# Running cellpose locally with a GPU

Notebook version to run cellpose locally with GPU access on windows and Linux systems, the code cells that are going to be executed are indicated with [ ] at the left, run them with the play button or ctrl + enter

## Setting up your notebook

We are going to setup everything in order to run the cellpose analysis. 

Check CUDA version and GPU

In [ ]:
!nvcc --version
!nvidia-smi

Now we are going to import the libraries and check GPU (the first time you import cellpose the models will download).

In [ ]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import tifffile as tf
import skimage.io
import zarr

from urllib.parse import urlparse
from cellpose import models, core

use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

# call logger_setup to have output of cellpose written
from cellpose.io import logger_setup
logger_setup();


### Load your images

Here you can put the files that are going to be processed, just separate them with a ",".
As a guide you can copy the paths of several files in windows selecting them in the file explorer and *Shift + Rigth click* + *Copy as path*.
They are copied with windows filepath structure, in order to be listed, just add an r before the ".

In [ ]:
# REPLACE FILES WITH YOUR IMAGE PATHS
files = [r"I:\Example_cellpose.tif"]
    
imgs = [tf.imread(f) for f in files]
nimg = len(imgs)

## Run cellpose 3D

There are two ways to run cellpose in 3D, this cells shows both, choose which one works best for you.

First way: computes flows from 2D slices and combines into 3D flows to create masks (Not so goog for me)

The only thing that have to be changed is the model type nad the chanels in [A,B]; A should be your cytoplasm channel and B your nuclei channel. If there is no nuclei channel, just left 0.

Also, remember to setup your estimated diameter in pixels.

In [ ]:
# DEFINE CELLPOSE MODEL
# model_type='cyto', 'nuclei', 'cyto2' or 'omni'
model = models.Cellpose(gpu=use_GPU, model_type='cyto2')

# * with 3D you have to set the diameter manually (no auto detect) *

### TWO WAYS TO RUN CELLPOSE IN 3D

# 1. computes flows from 2D slices and combines into 3D flows to create masks
print('running cellpose 2D slice flows => masks')
masks, flows, styles, diams = model.eval(imgs, channels=[0,0], diameter=15, do_3D=True)


Second way: computes masks in 2D slices and stitches masks in 3D based on mask overlap

Note: Stitching (with stitch_threshold > 0) can also be used to track cells over time.

In [ ]:
# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=use_GPU, model_type='cyto2')

# 2. computes masks in 2D slices and stitches masks in 3D based on mask overlap
print('running cellpose 2D + stitching masks')
masks_stitched, flows_stitched, styles_stitched, _ = model.eval(imgs, channels=[1,2], diameter=40, do_3D=False, stitch_threshold=0.5)

### Save output

You will see the files saved in the folder where you have your original file.

You can choose to save the results as Tif files or Numpy arrays.

In [ ]:
#SAVE AS TIFF

from cellpose import io
io.save_masks(imgs, masks_stitched, flows_stitched, files, tif=True, save_flows=True)

In [ ]:
#SAVE AS NUMPY

from cellpose import io
io.masks_flows_to_seg(imgs, masks_stitched, flows_stitched, diams, files)

### Inspect the results in 3D using Napari

Load and open Napari viewer

In [ ]:
import napari
viewer = napari.Viewer()

Now you can drag ans drop your files to the viewer.

If you have few files, you can run the next cell to load them in Napari.